In [3]:
#from __future__ import annotations

#import random
#import shutil
#from datetime import datetime
from pathlib import Path
from typing import Callable, Dict, Iterator, List, Optional, Sequence, Tuple, Union

#import numpy as np
import tensorflow as tf
import torch
from loguru import logger
#from torch.nn.utils.rnn import pad_sequence
#from tqdm import tqdm
from scipy.io import arff

Tensor = torch.Tensor

2022-06-07 12:18:09.005904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-07 12:18:09.005940: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
def get_eeg(data_dir: Path="../data/raw") -> Path:
    dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00264/EEG%20Eye%20State.arff"  # noqa: E501
    datapath = tf.keras.utils.get_file(
        "eeg_data", origin=dataset_url, untar=False, cache_dir=data_dir
    )

    datapath = Path(datapath)
    logger.info(f"Data is downloaded to {datapath}.")
    return datapath

In [5]:
from __future__ import annotations
from typing import Tuple
from tqdm import tqdm
import random
Tensor = torch.Tensor


class BaseDataset:
    def __init__(self, datapath: Path) -> None:
        self.path = datapath
        self.data =  self.process_data()

    def process_data(self) -> None:
        data = arff.loadarff(self.path)
        cur_label = int(data[0][0][14]) #index 14 = label
        EEG_list = [] #Lege lijst waarin meerdere observaties worden opgeslagen
        EEG_full = [] #Lege lijst waarin meerdere batches in worden samengevoegd.
        for record in data[0]:
            if int(record[14]) == cur_label:
                EEG_dim = [] #Lege lijst waarin de EEG_dim van een bepaalde observatie in kunnen worden opgeslagen.
                for index, i in enumerate(record):
                    if index != 14:
                        EEG_dim.append(i)
                EEG_dim = torch.Tensor(EEG_dim)
                EEG_list.append(EEG_dim)
            else:
                EEG_full_label = (cur_label, torch.stack(EEG_list))
                EEG_full.append(EEG_full_label)
                cur_label = int(record[14])
                EEG_list = [] #Lege lijst waarin meerdere observaties in kunnen worden opgeslagen.
                EEG_dim = [] #Lege lijst waarin de EEG_dim van een bepaalde observatie in kunnen worden opgeslagen.
                for index, i in enumerate(record):
                    if index != 14:
                        EEG_dim.append(i)
                EEG_dim = torch.Tensor(EEG_dim)
        EEG_full_label = (cur_label, torch.stack(EEG_list))
        EEG_full.append(EEG_full_label)
        return EEG_full

    def __len__(self) -> int:
        return len(self.data)
   
    def __getitem__(self, idx: int) -> Tuple:
        return self.data[idx]

In [13]:
dataloader = BaseDataset(datapath=get_eeg())

2022-06-07 13:00:30.360 | INFO     | __main__:get_eeg:8 - Data is downloaded to ../data/raw/datasets/eeg_data.


In [ ]:



class BaseDataIterator (BaseDataset):
    def __init__(self, datapath):
        self.data = datapath
       
        self.dataset = dataset
        self.batchsize = batchsize
    def __len__(self) -> int:
        # the lenght is the amount of batches
        return int(len(self.dataset) / self.batchsize)

    def proces_data(self)
     data = arff.loadarff(datapath)

    def __iter__(self) -> BaseDataIterator:
        # initialize index
        self.index = 0
        self.index_list = torch.randperm(len(self.dataset))
        return self
    
    def batchloop(self) -> Tuple[Tensor, Tensor]:
        X = []  # noqa N806
        Y = []  # noqa N806
        # fill the batch
        for _ in range(self.batchsize):
            x, y = self.dataset[int(self.index_list[self.index])]
            X.append(x)
            Y.append(y)
            self.index += 1
        return X, Y

    def __next__(self) -> Tuple[Tensor, Tensor]:
        if self.index <= (len(self.dataset) - self.batchsize):
            X, Y = self.batchloop()
            return X, Y
        else:
            raise StopIteration

class PaddedDatagenerator(BaseDataIterator):
     def __init__(self, dataset: BaseDataset, batchsize: int) -> None:
         # we initialize the super class BaseDataIterator
         # we now have everything the BaseDataIterator can do, for free
         super().__init__(dataset, batchsize)
    
     def __next__(self) -> Tuple[Tensor, Tensor]:
         if self.index <= (len(self.dataset) - self.batchsize):
             X, Y = self.batchloop()
             # we just want to add padding
             X_ = pad_sequence(X, batch_first=True, padding_value=0)  # noqa N806
             return X_, torch.tensor(Y)
         else:
             raise StopIteration


In [14]:
def window(x: Tensor, n_time: int) -> Tensor:
    """
    Generates and index that can be used to window a timeseries.
    E.g. the single series [0, 1, 2, 3, 4, 5] can be windowed into 4 timeseries with
    length 3 like this:

    [0, 1, 2]
    [1, 2, 3]
    [2, 3, 4]
    [3, 4, 5]

    We now can feed 4 different timeseries into the model, instead of 1, all
    with the same length.
    """
    n_window = len(x) - n_time + 1
    time = torch.arange(0, n_time).reshape(1, -1)
    window = torch.arange(0, n_window).reshape(-1, 1)
    idx = time + window
    return idx

In [18]:
window_size = 1
idx = window(dataloader, window_size)
idx.shape()


TypeError: 'torch.Size' object is not callable

In [ ]:
dataloader.__getitem_

In [7]:
dataloader = BaseDataset(datapath=get_eeg())

2022-06-07 12:18:25.516 | INFO     | __main__:get_eeg:8 - Data is downloaded to ../data/raw/datasets/eeg_data.


In [7]:
dataloader.__len__()

24

In [8]:
dataloader.__getitem__(23)

(1,
 tensor([[4304.1001, 4016.9199, 4273.8501, 4145.1299, 4340.0000, 4623.5898,
          4084.1001, 4646.6699, 4223.0801, 4229.2300, 4182.0498, 4282.5601,
          4606.1499, 4364.1001],
         [4303.0801, 4016.9199, 4270.7700, 4138.9702, 4342.0498, 4621.5400,
          4082.5601, 4645.6401, 4220.0000, 4229.2300, 4180.5098, 4276.9199,
          4602.0498, 4362.5601],
         [4304.6201, 4018.4600, 4272.3101, 4139.4902, 4340.0000, 4619.4902,
          4082.5601, 4637.9502, 4203.0801, 4226.6699, 4179.4902, 4278.4600,
          4602.0498, 4360.0000],
         [4301.0298, 4013.8501, 4268.7202, 4140.0000, 4341.0298, 4617.4399,
          4082.0500, 4626.1499, 4196.4102, 4224.1001, 4177.9502, 4278.9702,
          4603.0801, 4360.0000],
         [4300.0000, 4009.2300, 4263.5898, 4138.4600, 4342.5601, 4617.9502,
          4083.5901, 4623.0801, 4205.6401, 4226.6699, 4177.9502, 4277.4399,
          4601.0298, 4362.0498],
         [4302.5601, 4008.7200, 4265.1299, 4138.9702, 4336.4102, 4618.4